In [1]:
import pandas as pd
from collections import Counter

#bokeh

from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.models import HoverTool
from bokeh.core.properties import value
from bokeh.io import output_file, show, save
from bokeh.plotting import figure
from IPython.display import display, HTML
from bokeh.transform import factor_cmap, factor_mark



#custom
from py.helper import DataProcessing
from py.ml_metrics import plot_confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans


In [2]:
orf1 = DataProcessing('coronavirus_orf1ab.fasta', 'coronavirus_orf1ab_meta.csv')

In [3]:
# read for data folder and out put 
offset, kmer = 4, 4 # number of bases to offset
amino_df = orf1.get_amino_df(offset, 'csv')
print(f"shape WITH duplicates: {amino_df.shape}")

# remove duplicates
amino_df.drop_duplicates(subset='Accession', keep=False, inplace=True)
print(f"shape WITHOUT duplicates: {amino_df.shape}")
amino_df.head()

shape WITH duplicates: (3046, 13)
shape WITHOUT duplicates: (2384, 13)


,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,GenBank_Title,seq_offset_0,seq_offset_1,seq_offset_2,seq_offset_3
1,YP_009555238,2019-02-21T00:00:00Z,Betacoronavirus 1,7095,USA,NaN,NaN,NaN,Orf1ab [Human coronavirus OC43],MSKINKYGLELHWAPEFPWMFEDAEEKLDNPSSSEVDMICSTTAQK...,SKINKYGLELHWAPEFPWMFEDAEEKLDNPSSSEVDMICSTTAQKL...,KINKYGLELHWAPEFPWMFEDAEEKLDNPSSSEVDMICSTTAQKLE...,INKYGLELHWAPEFPWMFEDAEEKLDNPSSSEVDMICSTTAQKLET...
2,YP_002308478,2018-08-24T00:00:00Z,Bulbul coronavirus HKU11,6264,Hong Kong,Pycnonotus jocosus,NaN,2007-01,orf1ab polyprotein [Bulbul coronavirus HKU11-934],MVKNVSKRSPIVLPQIQPPPLQLFIAVAAAEEGHPKDLKYLGNYNL...,VKNVSKRSPIVLPQIQPPPLQLFIAVAAAEEGHPKDLKYLGNYNLV...,KNVSKRSPIVLPQIQPPPLQLFIAVAAAEEGHPKDLKYLGNYNLVT...,NVSKRSPIVLPQIQPPPLQLFIAVAAAEEGHPKDLKYLGNYNLVTS...
3,YP_009513008,2018-08-24T00:00:00Z,Hedgehog coronavirus 1,7150,Germany,Erinaceus europaeus,feces,2012,orf1ab [Betacoronavirus Erinaceus/VMC/DEU/2012],MSSATGEGSQGARATYRAALNNEKRHDHVALTVPCCGTEAKVTALS...,SSATGEGSQGARATYRAALNNEKRHDHVALTVPCCGTEAKVTALSP...,SATGEGSQGARATYRAALNNEKRHDHVALTVPCCGTEAKVTALSPW...,ATGEGSQGARATYRAALNNEKRHDHVALTVPCCGTEAKVTALSPWF...
4,YP_009513020,2018-08-24T00:00:00Z,Coronavirus HKU15,6267,China: Hong Kong,Sus scrofa,NaN,2010,replicase polyprotein [Porcine coronavirus HKU15],MAKNKSKRDAIALPENVPPPLQLFIHVAAAEEGHPKVTTYLGNYNL...,AKNKSKRDAIALPENVPPPLQLFIHVAAAEEGHPKVTTYLGNYNLY...,KNKSKRDAIALPENVPPPLQLFIHVAAAEEGHPKVTTYLGNYNLYA...,NKSKRDAIALPENVPPPLQLFIHVAAAEEGHPKVTTYLGNYNLYAT...
5,YP_009389424,2017-07-14T00:00:00Z,Wencheng Sm shrew coronavirus,6324,China,Suncus murinus,NaN,2015,ORF1ab polyprotein [Wencheng Sm shrew coronavi...,MSVSKVELFVPISDEVDATHFGTFGDAVEAYASAAPSFEGVYFVAY...,SVSKVELFVPISDEVDATHFGTFGDAVEAYASAAPSFEGVYFVAYG...,VSKVELFVPISDEVDATHFGTFGDAVEAYASAAPSFEGVYFVAYGL...,SKVELFVPISDEVDATHFGTFGDAVEAYASAAPSFEGVYFVAYGLQ...


# Check the hosts

In [4]:

from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

counts = Counter(amino_df['Species'])
data = {'Species':list(counts.keys()), 'Count':list(counts.values())}
source = ColumnDataSource(data=data)

print(f"Total number of viral species:{len(counts)}")
mapper = linear_cmap(field_name='Count', palette=Spectral6 ,low=min(data['Count']) ,high=max(data['Count']))

p = figure(x_range=data['Species'], plot_height=400, plot_width=750, title="Virus Species",
           toolbar_location='right')
pv = p.vbar(x='Species', top='Count', width=0.9, source=source, color=mapper)
p.add_tools(HoverTool(
#     renderers=[pv],
    tooltips=[
        ( 'Species',   '@'+'Species'            ), #state
        ( 'Count',  '@'+'Count'            ), #count
    ],

    formatters={
        
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
              ))

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.visible = False
p.yaxis.axis_label = 'Number of sequences'
# output_file("bars.html")
# save(p)
# display(HTML('bars.html'))


Loading BokehJS ...

Total number of viral species:84


In [5]:
# x = np.linspace(0, 2*np.pi, 2000)
# y = np.sin(x)

In [6]:
# p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
#            background_fill_color='#efefef')
# r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [7]:
def update(Species, Show_All):
    if not Show_All:
        c_ount = Counter(amino_df['Species'])
        pv.data_source.data = {'Species':[Species], 'Count':[c_ount[Species]]}
        print("> Check 'Show_All' to see the entire dataset.")
    else:
        counts = Counter(amino_df['Species'])
        pv.data_source.data = {'Species':list(counts.keys()), 'Count':list(counts.values())}
        print("> Uncheck 'Show_All' to see the species level data.")
    push_notebook()

# An interactive bar chart of the coronavirus 'orf1ab' sequence counts in the dataset

In [8]:

# interact(update, f=amino_df['Species'].unique(), w=(0,50), A=(1,10), phi=(0, 20, 0.1))
show(p, notebook_handle=True)
fn = interact(update, Species=sorted(amino_df['Species'].unique()), Show_All=False)


interactive(children=(Dropdown(description='Species', options=('Alphacoronavirus 1', 'Alphacoronavirus 2', 'Al…